Author: Johannes Peter Knoll

(For josef)

# Introduction

To determine the RRI, my program first checks where the ECG is processable: \
key: "valid_ecg_regions" - list of tuples, e.g.: 
\[ \[start_valid_region_1, end_valid_region_1\], \[..._2,..._2\], ... \] 

It then calculates the position of the r-peaks within these regions (but adds them to the same list): \
key: "hamilton" - list of values, e.g.: \[ rpeak_position_1, rpeak_position_2, ... \]

From those, it can determine the RRI (separated into lists, containing the rri values within a valid ecg region): \
key: "RRI" - list of lists, e.g.: \[ \[\<rri_values_within_valid_ecg_region_1\>\], \[\<..._2\>\], ... \]

I also found a channel in the provided SHIP data which was called: "RR". I assumed it might also hold the 
RR-Intervals, which is why I added this to the results as well: \
key: "SHIP_RR" - list of values, e.g.: \[ ... \]

These results were saved as individual dictionaries for every .edf-file in one .pkl file in the following 
format:

In [ ]:
# {
#     "file_name":     
#         Name of the (.edf) file the results are calculated for,

#     "valid_ecg_regions": 
#         List of valid regions in the ECG data, that is used during r-peak detection,
    
#     "rpeak-function-name":
#         List of r-peak locations detected by the rpeak-function-name function.
    
#     "RRI":
#         List of RR-intervals calculated from the r-peak locations.
    
#     "RRI_frequency":
#         Sampling frequency of the RR-intervals.
    
#     "SHIP_RR":
#         Already provided RR-intervals for the SHIP data (?)

#     "SHIP_RR_frequency":
#         Sampling frequency of the SHIP_RR intervals.
# }

In [ ]:
import pickle

def load_from_pickle(file_path: str):
    """
    Load data from a pickle file as a generator.

    ARGUMENTS:
    --------------------------------
    file_path: str
        path to the pickle file
    
    RETURNS:
    --------------------------------
    generator
        data from the pickle file
    """

    with open(file_path, "rb") as f:
        while True:
            try:
                yield pickle.load(f)
            except:
                break


def append_to_pickle(data, file_path):
    """
    Append data to a pickle file, without deleting previous data.

    ARGUMENTS:
    --------------------------------
    data: any
        data to be saved
    file_path: str
        path to the pickle file
    
    RETURNS:
    --------------------------------
    None
    """
    with open(file_path, "ab") as f:
        pickle.dump(data, f)

# Split Results

In [ ]:
path_to_results = "Processed_SHIP/EDF-gek_Results.pkl"
path_to_new_results = "Processed_SHIP/SHIP_Results"

count_splits = 1
count = 0
results_generator = load_from_pickle(path_to_results)
for individual_file_results in results_generator:
    append_to_pickle(individual_file_results, path_to_new_results + str(count_splits) + ".pkl")
    count += 1
    if count > 150:
        count = 0
        count_splits += 1

# Fuse Results

In [ ]:
import os

path_to_directory = "<Path>"
path_to_directory = "Processed_SHIP/"

path_to_new_results = "<Path>/SHIP_Results.pkl"
path_to_new_results = "Processed_SHIP/SHIP_Results.pkl"

all_files = [file for file in os.listdir(path_to_directory) if file.endswith(".pkl")]

for file in all_files:
    results_generator = load_from_pickle(path_to_directory + file)
    for individual_file_results in results_generator:
        append_to_pickle(individual_file_results, path_to_new_results)
    os.remove(path_to_directory + file)

# Access Results

In [ ]:
path_to_results = "<Path>/SHIP_Results.pkl"

results_generator = load_from_pickle(path_to_results)

count = 0
first_entry = True
for individual_file_results in results_generator:
    if first_entry:
        for key in individual_file_results:
            print(key)
        first_entry = False

    count += 1

print(f"Total number of files: {count}")